# Deep Learning Models

This is not a deep learning course, but by request, I will introduce simple examples to get started in deep learning and highlight some aspects that I find interesting.

In [3]:
%matplotlib inline

In [4]:
import warnings
warnings.simplefilter('ignore', RuntimeWarning)

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [6]:
import tensorflow as tf
import tensorflow.keras as keras

## Built-in models

Keras comes with many built-in model architectures that serve as great starting points. Sometimes they even come pre-trained on massive data sets and can be used out-of-the-box, or improved with some transfer learning.

In [1]:
import tensorflow_datasets as tfds

In [101]:
ds, info = tfds.load(name='fashion_mnist', 
                     as_supervised=True, 
                     batch_size=32,
                     with_info=True)

In [102]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

#### Preprocesss the image to make it work InceptionV3

In [103]:
size = [75, 75]
X_train = (ds['train'].
           map(lambda image, label: 
               (tf.image.resize(tf.image.grayscale_to_rgb(image), size), label)))
X_test = (ds['test'].
          map(lambda image, label:
              (tf.image.resize(tf.image.grayscale_to_rgb(image), size), label)))

In [104]:
shape = [item[0].shape for item in X_train.take(1)][0]
shape

TensorShape([32, 75, 75, 3])

In [106]:
base_model = InceptionV3(
    input_shape=shape[1:],
    weights='imagenet', 
    include_top=False,
    classes=10
)

#### Freeze weights in base model

In [107]:
for layer in base_model.layers:
    layer.trainable = False

#### Add the last layers of the model to predict 10 classes

In [108]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

In [109]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [110]:
model.fit(X_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 105s 56ms/step - loss: 2.9949 - accuracy: 0.6888
Epoch 2/5
1875/1875 [==============================] - 114s 61ms/step - loss: 0.7069 - accuracy: 0.7416
Epoch 3/5
1875/1875 [==============================] - 111s 59ms/step - loss: 0.6888 - accuracy: 0.7509
Epoch 4/5
1875/1875 [==============================] - 112s 59ms/step - loss: 0.6708 - accuracy: 0.7560
Epoch 5/5
1875/1875 [==============================] - 111s 59ms/step - loss: 0.6756 - accuracy: 0.7447


In [111]:
test_loss, test_acc = model.evaluate(X_test)
test_acc

313/313 [==============================] - 16s 51ms/step - loss: 0.6624 - accuracy: 0.7603


0.7602999806404114

## Building your own model

### Sequential model

The simplest way to build a model